##### 小批量爬取数据

In [4]:
import redis as redis
import requests
import random
import numpy as np
import re
import os
import selenium.webdriver as webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait #等待页面加载某些元素
from selenium.webdriver.support import expected_conditions as EC

import smallLib  as helper

pool = redis.ConnectionPool(host='127.0.0.1',port=6379,password='123456',decode_responses=True)   
r = redis.Redis(connection_pool=pool)

In [5]:

origin_table = 'demo_targets1'
names = r.hkeys(origin_table)
urls = r.hvals(origin_table)
pairs = dict(zip(names,urls))
temp_table = 'demo_target_temp'
#r.delete(temp_table)
#r.hmset(temp_table, pairs) 
store_table='demo_target_small_batch'

- checkpoint1 从用户主页抓取用户视频链接
    - 使用selenium模拟访问，滑动页面到底端时可以抓取该用户的全部视频
    - 抓取结果缓存进redis，可以使用helper.redis_export(table,name) 导出为json

In [6]:

targets = {}
pair = helper.get_user(temp_table)
'''
while len(pair) > 1:
    pair= helper.get_user(r,temp_table)
    name = pair[0]
    url = pair[1]
    current_length = len(r.hget('demo_target_small_batch', name).split(','))
    try :
        href =  helper.get_all_video_form_homepage(url)
        if len(href)>0 :
            #targets[name] = href
            if len(href)>current_length:
                r.hset('demo_target_small_batch', name, str(href))
            helper.delete_user(r,temp_table,name=name)
    except Exception:
        continue


'''


- 统计抓取视频数

In [7]:
batch = r.hgetall(store_table)
cnt= 0
for val in batch.values():
    cnt += len(val.split(','))
#print(cnt)